In [83]:
import os
import logging
import optuna
import pandas as pd
from copy import deepcopy
from sklearn.metrics import mean_pinball_loss
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import joblib

logging.basicConfig(level=logging.INFO)

# Set paths
BASE_PATH = os.getenv("BASE_PATH", "/Users/florian/Documents/github/DP2/Energy_production_price_prediction/")
DATA_PATH = os.path.join(BASE_PATH, "Generation_forecast/Solar_forecast/data/train_norm.csv")
MODEL_SAVE_PATH = os.path.join(BASE_PATH, "Generation_forecast/Solar_forecast/models/rnn")

# Load data
data = pd.read_csv(DATA_PATH)
df = deepcopy(data)

# Drop NaN values
df.dropna(inplace=True)

# Separate features and target
X = df.drop(columns="Target_Capacity_MWP_%").values
y = df["Target_Capacity_MWP_%"].values

# Load test data
df_api = pd.read_csv(os.path.join(BASE_PATH, "Generation_forecast/Solar_forecast/data/test_norm.csv"))
X_api = df_api.drop(columns="Target_Capacity_MWP_%").values
y_api = df_api["Target_Capacity_MWP_%"].values

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_api)

#joblib.dump(scaler, os.path.join(MODEL_SAVE_PATH, "scaler.pkl"))

In [58]:
class RNN_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.3):
        super(RNN_Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )

        self.fc = nn.Linear(
            in_features=hidden_size,
            out_features=output_size
        )

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [59]:
def pinball_loss(y_true, y_pred, quantiles):
    errors = y_true - y_pred
    quantiles = quantiles.view(1, -1).expand_as(y_pred)  # (batch_size, 9)
    loss = torch.max((quantiles - 1) * errors, quantiles * errors)
    return torch.mean(loss)

In [84]:
# Previous imports and data preprocessing remain the same
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_api, dtype=torch.float32).view(-1, 1)

# Create TensorDataset and DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

# Create TensorDataset and DataLoader for testing
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = X_train_scaled.shape[1]  
hidden_size = 128                   
num_layers = 3                       
output_size = 9                       
dropout = 0.3                         
learning_rate = 0.0001               
batch_size = 32                       
num_epochs = 500                      
patience = 10                         
rel_improvement_threshold = 0.00000001     

# Quantiles (9 quantile levels)
quantiles = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype=torch.float32)

model = RNN_Model(input_size, hidden_size, num_layers, output_size, dropout)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [85]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, 9) where 9 is the number of quantiles

        # Expand y_batch to match y_pred dimensions
        y_batch_expanded = y_batch.repeat(1, len(quantiles))  # (batch_size, 9)

        # Compute the loss
        loss = pinball_loss(y_batch_expanded, y_pred, quantiles) 

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.7f}')

    # Relative stopping
    rel_improvement = (previous_loss - epoch_loss) / previous_loss
    if rel_improvement < rel_improvement_threshold:
        print(f"Relative improvement below threshold. Stopping training.")
        break
    previous_loss = epoch_loss

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
model.load_state_dict(best_model)

# Test the model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, 9)

        # Expand y_batch to match y_pred dimensions
        y_batch_expanded = y_batch.repeat(1, len(quantiles))  # (batch_size, 9)

        # Compute the loss
        loss = pinball_loss(y_batch_expanded, y_pred, quantiles) 
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.7f}')

Epoch [1/500], Loss: 0.0076812
Epoch [2/500], Loss: 0.0058111
Epoch [3/500], Loss: 0.0055921
Epoch [4/500], Loss: 0.0053852
Epoch [5/500], Loss: 0.0052108
Epoch [6/500], Loss: 0.0050388
Epoch [7/500], Loss: 0.0049271
Epoch [8/500], Loss: 0.0048123
Epoch [9/500], Loss: 0.0047298
Epoch [10/500], Loss: 0.0047203
Epoch [11/500], Loss: 0.0046854
Epoch [12/500], Loss: 0.0046827
Epoch [13/500], Loss: 0.0046680
Epoch [14/500], Loss: 0.0046699
Relative improvement below threshold. Stopping training.
Test Loss: 0.0058946


In [7]:
#joblib.dump(model, os.path.join(MODEL_SAVE_PATH, "rnn_model.pkl"))

In [88]:
import numpy as np

all_predicitons = []

model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)
    
        batch_predictions = model(X_batch)
        all_predicitons.append(batch_predictions.cpu().numpy()) 

predictions = np.concatenate(all_predicitons, axis=0)

In [91]:
import plotly.graph_objects as go

# Plot the predictions vs. true targets with Plotly
y_test_numpy = y_api
loss_all_quantiles = []

fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test_numpy.flatten(), mode='lines', name='True Targets', marker=dict(color='black')))
for i in range(0, 9):
    y_pred = predictions[:, i]
    loss_q = mean_pinball_loss(y_test_numpy, y_pred, alpha= 0.1*(i+1)) * 2281.8743117295026
    fig.add_trace(go.Scatter(y=y_pred, mode='lines', name=f'Quantile 0.{i+1}, {loss_q:.4f}'))
    loss_all_quantiles.append(loss_q)

fig.update_layout(title='Predictions vs. True Targets',
                  xaxis_title='Data points',
                  yaxis_title='Target Capacity MWP %')

fig.show()
print((sum(loss_all_quantiles)/9).round(3))

13.478
